<a href="https://colab.research.google.com/github/BlackCurrantDS/DeepLearning/blob/main/IDL_Assignments6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import numpy as np
import keras

In [11]:
!python prepare_data2.py shk_input.txt skp \\n -m 500

2020-12-01 22:01:03.853965: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Split input into 40000 sequences...
Longest sequence is 65 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
40000 sequences remaining.
Longest remaining sequence has length 65.
Removing length-0 sequences...
40000 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
S

In [12]:
#Getting the data
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(parse_seq) #since changing seq lengths

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'E': 3, 'h': 4, 'f': 5, 'o': 6, '3': 7, 'F': 8, '!': 9, 'J': 10, 's': 11, 'r': 12, 'L': 13, 't': 14, 'k': 15, 'H': 16, 'B': 17, 'X': 18, 'V': 19, 'y': 20, 'C': 21, 'e': 22, 'M': 23, '\n': 24, 'N': 25, ':': 26, 'K': 27, 'v': 28, 'd': 29, 'T': 30, 'i': 31, 'm': 32, 'l': 33, 'P': 34, 'u': 35, ',': 36, 'w': 37, '.': 38, 'a': 39, 'S': 40, 'I': 41, 'b': 42, 'U': 43, 'c': 44, 'Q': 45, 'q': 46, 'R': 47, 'G': 48, "'": 49, 'Z': 50, 'W': 51, 'p': 52, '?': 53, ' ': 54, 'j': 55, '-': 56, 'x': 57, 'z': 58, '$': 59, 'O': 60, ';': 61, 'A': 62, 'g': 63, '&': 64, 'n': 65, 'D': 66, 'Y': 67, '<PAD>': 0, '<S>': 1, '</S>': 2}
68


In [13]:
#one hot encoding

def hotcoding(input_data):
  
  one_hot_coded_data = tf.one_hot(indices = input_data, depth = vocab_size)
  return one_hot_coded_data;

one_hot_coded_data = data.map(hotcoding)

In [14]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

one_hot_coded_data = one_hot_coded_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [15]:
# Number of RNN units
rnn_units = 512

In [25]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in one_hot_coded_data.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape,  (batch_size, sequence_length, vocab_size))

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           17408     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 68)            69700     
Total params: 5,334,084
Trainable params: 5,334,084
Non-trainable params: 0
_________________________________________________________________


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

NameError: ignored